# Can be ignored, we were using to run some tests whether 2 datas are equal or not

In [1]:
import numpy as np
from goldcodegenerator import PRN

In [2]:
filepath = '/home/joel/gps-sdr-sim-master/gen_data_split/output_upd.dat'
# filepath = '/home/joel/prn_10_removed_data/split/mydataa'
data1 = np.fromfile(filepath,dtype = np.short)
compdata1 = data1[::2] + (1j)*data1[1::2]
# print(compdata1[25323923])

filepath = '/home/joel/gain0.5_data/split/mydataa'
data = np.fromfile(filepath,dtype = np.short)
compdata2 = data[::2] + (1j)*data[1::2]
# print(compdata2[120323923])

filepath = '/home/joel/gps-sdr-sim-master/gen_data_split/mydataa'
data2 = np.fromfile(filepath,dtype = np.short)
compdata3 = data2[::2] + (1j)*data2[1::2]

# print(compdata1[25323923])

del data1
del data
del data2



<class 'numpy.ndarray'>


In [4]:
print(type(np.real(compdata3[1])))

<class 'numpy.float64'>


In [3]:
subfrm3 = 48324055
compdata1 = compdata1[:subfrm3+700000]
compdata2 = compdata2[:subfrm3+700000]
compdata3 = compdata3[:subfrm3+700000]
# print(np.abs(compdata1[subfrm3:subfrm3+640000] - compdata2[subfrm3:subfrm3+640000]))
sv = 10
prn = PRN(sv)
prn = [-1 if x == 0 else x for x in prn]

prn_upsample = []
n = 0
for i in range(4000):
    if ((i+1)/4000 <= (n+1)/1023):
        prn_upsample.append(prn[n])
    else:
        n = n+1
        prn_upsample.append(prn[n])
# Doppler Shift
doppler = {18 : 0, 12: 2697.905, 13: -2848.802, 10: 2891.328, 5: -2449.682, 15: -1816.517, 23: 2137.817}
gold_code = []
for i in range(20):
  gold_code.append(prn_upsample)
gold_code = np.array(gold_code).flatten()
code = [1,0,0,0,1,0,1,1]
v1 = []
for i in range(len(code)):
  if code[i] == 1:
    v1.append(np.array(gold_code))
  else:
    v1.append(np.multiply(-1,np.array(gold_code)))
v1 = np.array(v1).flatten()



In [4]:
f_d = doppler[sv] 
# f_d = 1000
n = np.arange(len(compdata2[:subfrm3+700000]))
compdata1_shifted = np.round(np.multiply(compdata1,np.exp(-1j*2*np.pi*f_d*n/4000000)), decimals=5)

In [5]:
compdata2_shifted = np.round(np.multiply(compdata2,np.exp(-1j*2*np.pi*f_d*n/4000000)), decimals=5)

In [6]:
compdata3_shifted = np.round(np.multiply(compdata3,np.exp(-1j*2*np.pi*f_d*n/4000000)), decimals=5)

In [10]:
del compdata1
del compdata2
del compdata3

In [17]:
ind = np.where(np.imag(compdata3_shifted) == max(np.imag(compdata3_shifted)))[0]
print(compdata3_shifted[ind])

print(compdata3_shifted[:100])

[68.07283+1399.42384j]
[  703.     +6.0000000e+00j    78.0219 +4.6457000e+00j
   230.01776+9.1073000e-01j   230.04677+1.8658700e+00j
   534.91172-9.7186800e+00j  -117.1969 -7.3407600e+00j
  -270.17223-2.6396600e+00j  -624.8749 +1.3869570e+01j
  -624.76913+1.7706740e+01j   222.65028-1.3109290e+01j
  -129.6388 -1.1125710e+01j   183.17118-2.1173580e+01j
    30.8995 -2.6871900e+00j   136.46624-1.3075330e+01j
   639.27858-3.1684290e+01j   435.91825-3.0745420e+01j
   586.35653-5.7723620e+01j   234.51422-5.4250170e+01j
   234.1021 -5.7308020e+01j   476.2041 -9.9411550e+01j
   326.368  -7.6921590e+01j   321.19426-4.4791190e+01j
   319.9925 -4.6149730e+01j    77.99191-4.1547800e+00j
   -85.90975-3.6761400e+00j  -358.5361 +5.0949620e+01j
  -306.46137+7.3616760e+01j  -306.24619+7.4015350e+01j
    58.61428+2.4766240e+01j   -92.93595+2.9460290e+01j
  -295.46994+3.1424760e+01j  -641.47123+1.1313560e+02j
  -236.60188-7.4245000e-01j  -236.60281+3.3212000e-01j
  -237.21408-2.5457100e+00j  -147.13688-2.

In [7]:
print(np.correlate(compdata1_shifted[subfrm3:subfrm3+640000], v1))
print(np.correlate(compdata2_shifted[subfrm3:subfrm3+640000], v1))
print(np.correlate(compdata3_shifted[subfrm3:subfrm3+640000], v1))

[1219066.60587-5362595.0945j]
[-4926519.32991+12963214.03881001j]
[-12194937.00832001+53644480.58536001j]


In [8]:
y1 = compdata1_shifted[subfrm3:subfrm3+640000]
y2 = compdata2_shifted[subfrm3:subfrm3+640000] 
y3 = compdata3_shifted[subfrm3:subfrm3+640000]
alp_est1 = np.dot(v1.T, y1)/np.dot(v1.T,v1)
alp_est2 = np.dot(v1.T, y2)/np.dot(v1.T,v1)
alp_est3 = np.dot(v1.T, y3)/np.dot(v1.T,v1)
print(alp_est1)
print(alp_est2)

print(alp_est3)

(1.9047915716718782-8.379054835156252j)
(-7.697686452984381+20.255021935640634j)
(-19.05458907550001+83.81950091462501j)


In [11]:
print(np.abs(alp_est3)/np.abs(alp_est2))
print(128/13)

3.9669740956217483
9.846153846153847
